In [0]:
# covid_lakehouse_student_todo.py
# -------------------------------------------------------------------
# Student Exercise: Build the COVID Lakehouse
# Fill in the TODO sections below to complete Bronze→Silver→Gold pipeline.
# -------------------------------------------------------------------

# ───────────────────────────────────────────────────────────────────────────
# Step 1: Bootstrap Spark + Hive Metastore & Prep Workspace
# ───────────────────────────────────────────────────────────────────────────
from pyspark.sql import SparkSession
import shutil, os

# TODO: Launch SparkSession with Hive support
spark = (
    SparkSession.builder
      .appName("COVID_Lakehouse_StudentExercise")
      .enableHiveSupport()
      .getOrCreate()
)

# TODO: Create & switch to your Hive database
# e.g. spark.sql("CREATE DATABASE IF NOT EXISTS covid")
#      spark.sql("USE covid")

# TODO: Define & clean your workspace folder (DBFS or local)
# e.g. base = "dbfs:/tmp/COVID_exercise"
#      dbutils.fs.rm(base, recurse=True)

print("✅ Spark & Hive metastore ready, workspace cleaned.")


# ───────────────────────────────────────────────────────────────────────────
# Step 2: Read CSVs & Bronze Ingestion
# ───────────────────────────────────────────────────────────────────────────
from pyspark.sql.functions import lit, col, current_timestamp, to_date

# TODO: Specify the path to the CSV dataset
# e.g. data_dir = "dbfs:/databricks-datasets/COVID/covid-19-data"

# TODO: Read each CSV into a DataFrame with header & inferSchema
# counties_df = spark.read.option(...).csv(...)
# recent_df   = spark.read.option(...).csv(...)
# states_df   = spark.read.option(...).csv(...)

# TODO: Show a few rows + schema for each DataFrame
# e.g. counties_df.show(5); counties_df.printSchema()

# Bronze paths under base
# TODO: set bronze_counties_path, bronze_recent_path, bronze_states_path

# --- Bronze | counties (full history) ---
# TODO: Drop existing table & directory, then CREATE TABLE bronze_counties USING DELTA LOCATION ...
# TODO: Write counties_df with .withColumn("source","..."), .withColumn("ingest_ts",current_timestamp()) to bronze_counties_path

# --- Bronze | recent (7-day slice) ---
# TODO: Repeat for bronze_recent

# --- Bronze | states (state-level) ---
# TODO: Repeat for bronze_states

print("✅ Bronze tables created and ingested.")


# ───────────────────────────────────────────────────────────────────────────
# Step 3: Create Silver 1 + Checkpoint
# ───────────────────────────────────────────────────────────────────────────
from delta.tables import DeltaTable

# TODO: Define silver1_path under base
# TODO: Drop silver_county table if exists, then CREATE TABLE silver_county (...) USING DELTA PARTITIONED BY (state) LOCATION ...
# TODO: Drop & recreate silver_county_ckpt table with schema (last_ts TIMESTAMP)

print("✅ Silver 1 skeleton created.")


# ───────────────────────────────────────────────────────────────────────────
# Step 4: Define & Run process_silver_county()
# ───────────────────────────────────────────────────────────────────────────
from datetime import datetime

def process_silver_county():
    """
    TODO: 1) Read last watermark from silver_county_ckpt
    TODO: 2) Union bronze_counties + bronze_recent
    TODO: 3) Filter for ingest_ts > last watermark
    TODO: 4) Clean & cast to report_date
    TODO: 5) MERGE into silver_county (whenMatchedUpdateAll, whenNotMatchedInsertAll)
    TODO: 6) Write new watermark back into silver_county_ckpt
    """
    # Your code here...
    pass

# TODO: Call process_silver_county() to populate Silver 1
# process_silver_county()

print("✅ Silver 1 upsert step complete.")


# ───────────────────────────────────────────────────────────────────────────
# Step 5: Build Silver 2 & Gold Tables
# ───────────────────────────────────────────────────────────────────────────
from pyspark.sql.functions import sum as _sum, window, avg, date_trunc

# --- Silver 2: Daily state roll-up ---
# TODO: Define silver2_path, drop existing table & directory
# TODO: CREATE TABLE silver_state_daily (...) PARTITIONED BY (report_date)
# TODO: Aggregate silver_county by report_date & state, write to silver_state_daily

# --- Gold 1: 7-day moving average ---
# TODO: Define gold1_path, drop existing table
# TODO: Read silver_state_daily, apply withWatermark + window of 7 days, compute avg, write to gold_state_ma

# --- Gold 2: Weekly summary ---
# TODO: Define gold2_path, drop existing table
# TODO: Compute weekly sums (date_trunc to week), write to gold_weekly_summary

print("✅ Silver 2 & Gold tables built.")
